In [10]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time

# URL 설정
base_url = "https://cointelegraph.com/tags/markets"
today = datetime.now()
half_months_ago = today - timedelta(days=180)

def create_driver():
    """Selenium WebDriver 생성 및 설정"""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Headless 모드 활성화
    options.add_argument("--disable-blink-features=AutomationControlled")  # Selenium 표시 제거
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_articles():
    """
    Cointelegraph Markets 섹션에서 기사를 크롤링하는 함수.
    
    Returns:
        articles (list): 크롤링된 기사 목록 (제목, 링크, 날짜 포함)
    """
    driver = create_driver()
    driver.get(base_url)
    
    # 페이지 로딩 대기
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li[data-testid='posts-listing__item']")))
    
    articles = []
    seen_titles = set()  # 중복 제거를 위한 제목 저장소
    
    # footer 요소 가져오기 (스크롤 제한용)
    footer_xpath = "//footer"
    footer_element = driver.find_element(By.XPATH, footer_xpath)
    
    try:
        while True:
            # 기사 추출
            elements = driver.find_elements(By.CSS_SELECTOR, "li[data-testid='posts-listing__item']")
            new_articles_found = False
            
            for element in elements:
                try:
                    # 제목과 링크 추출
                    title_tag = element.find_element(By.CLASS_NAME, "post-card-inline__title-link")
                    title = title_tag.text.strip()
                    link = title_tag.get_attribute("href")
                    
                    # 날짜 추출
                    date_tag = element.find_element(By.TAG_NAME, "time")
                    date_str = date_tag.get_attribute("datetime")
                    article_date = datetime.strptime(date_str[:10], '%Y-%m-%d')
                    
                    # 날짜 필터링 (180일 이내 기사만)
                    if half_months_ago <= article_date <= today and title not in seen_titles:
                        articles.append({
                            "title": title,
                            "link": link,
                            "date": article_date.strftime('%Y-%m-%d')
                        })
                        seen_titles.add(title)  # 제목 저장하여 중복 방지
                        new_articles_found = True
                        print(f"기사 저장: {title} ({article_date.strftime('%Y-%m-%d')})")
                    
                    # 일주일 범위를 벗어난 기사 발견 시 종료
                    elif article_date < half_months_ago:
                        print("180일 범위를 벗어난 기사가 발견되었습니다. 크롤링을 종료합니다.")
                        driver.quit()
                        return articles

                except Exception as e:
                    print(f"Error processing article: {e}")
            
            # 현재 스크롤 위치 확인
            current_scroll_position = driver.execute_script("return window.scrollY + window.innerHeight")
            footer_position = footer_element.location['y']  # footer의 Y축 위치
            
            # footer에 도달하기 전에만 스크롤 수행
            if current_scroll_position + 100 >= footer_position:
                print("Reached near the footer. Stopping scroll.")
                break
            
            # 더 이상 새로운 기사가 없으면 종료
            if not new_articles_found:
                print("더 이상 새로운 기사가 없습니다. 크롤링을 종료합니다.")
                break
            
            # 스크롤 내리기: `ActionChains`를 사용해 부드럽게 스크롤 수행
            actions = ActionChains(driver)
            actions.move_to_element(footer_element).scroll_by_amount(0, -200).perform()
            time.sleep(3)  # 페이지 로딩 대기
    
    except Exception as e:
        print(f"Error during scraping: {e}")
    
    finally:
        driver.quit()
    
    return articles

# 실행 코드
if __name__ == "__main__":
    print("Cointelegraph Markets 섹션에서 기사를 크롤링 중...")
    articles = scrape_articles()
    
    print(f"총 {len(articles)}개의 기사를 수집했습니다.")
    
    # CSV 파일로 저장
    with open("cointelegraph_articles.csv", mode="w", newline="", encoding="utf-8") as file:
        fieldnames = ["title", "link", "date"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()  # 헤더 추가
        writer.writerows(articles)  # 모든 데이터를 한 번에 저장

    print("기사 데이터가 'cointelegraph_articles.csv' 파일에 저장되었습니다!")

Cointelegraph Markets 섹션에서 기사를 크롤링 중...
기사 저장: Bitcoin reclaims $80K zone as BNB, TON, GT, ATOM hint at altcoin season (2025-03-16)
기사 저장: Bitcoin gets $126K June target as data predicts bull market comeback (2025-03-16)
기사 저장: Toncoin in 'great entry zone' as Pavel Durov's France exit fuels TON price rally (2025-03-16)
기사 저장: Forget Solana, XRP may flip Ethereum first amid 5-year high (2025-03-16)
기사 저장: Bitcoin’s role as an inflation hedge depends on where one lives — Analyst (2025-03-15)
기사 저장: Bitcoin poised to reclaim $90,000, according to derivatives metrics (2025-03-15)
기사 저장: Hyperliquid’s mystery 50x ETH whale is now betting on LINK (2025-03-14)
기사 저장: Growth in Bitcoin and stablecoin adoption could accelerate dedollarization (2025-03-14)
기사 저장: Bitcoin bull market in peril as US recession and tariff worries loom (2025-03-14)
기사 저장: Price analysis 3/14: BTC, ETH, XRP, BNB, SOL, ADA, DOGE, PI, LEO, LINK (2025-03-14)
기사 저장: Watch these Bitcoin price levels as BTC retests key $84

## csv 파일에 기사 내용 업데이트
csv 파일에 저장할 때, 이중 따옴표와 콤마를 공백으로, 줄바꿈을 띄어쓰기로 대체하면서 context 열에 기사 내용이 온전히 저장되도록 전처리

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import time

def create_driver():
    """Selenium WebDriver 생성 및 설정"""
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")  # Selenium 표시 제거
    options.add_argument("--headless")  # Headless 모드 활성화
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_article_content(link):
    """
    특정 링크를 방문하여 기사 내용을 크롤링하는 함수.

    Parameters:
        link (str): 기사의 URL
    
    Returns:
        content (str): 기사의 본문 내용
    """
    driver = create_driver()
    driver.get(link)
    
    try:
        # 기사 내용 대기 및 추출
        wait = WebDriverWait(driver, 10)
        article_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "post-content")))
        content = article_element.text.strip()
        
        # 줄바꿈 제거
        content = remove_newlines(content)
        
    except Exception:
        content = "내용을 가져올 수 없습니다."
    
    finally:
        driver.quit()
    
    return content

def remove_newlines(content):
    """
    텍스트에서 줄바꿈(\n)을 제거하는 함수.
    
    Parameters:
        content (str): 원본 텍스트
    
    Returns:
        str: 줄바꿈이 제거된 텍스트
    """
    return content.replace("\n", " ").strip()

def validate_row_fields(row, fieldnames):
    """
    데이터 딕셔너리의 필드를 검증하고 누락된 필드를 기본값으로 추가하는 함수.

    Parameters:
        row (dict): 데이터 딕셔너리
        fieldnames (list): CSV 파일의 필드 이름 목록

    Returns:
        dict: 검증된 데이터 딕셔너리
    """
    validated_row = {key: row.get(key, "").strip() for key in fieldnames}  # 누락된 필드는 빈 문자열로 채움
    return validated_row

def update_csv_with_content_in_batches(input_csv, batch_size=10):
    """
    기존 CSV 파일에 기사 내용을 100개씩 나누어 업데이트하는 함수.
    
    Parameters:
        input_csv (str): 입력 CSV 파일 경로 (제목, 링크, 날짜 포함)
        batch_size (int): 한 번에 처리할 기사 개수
    
    Returns:
        None: 기존 파일을 업데이트하여 저장
    """
    # 입력 CSV 파일 읽기
    with open(input_csv, mode="r", encoding="utf-8") as file:
        reader = list(csv.DictReader(file))
    
    # 필드 이름 정의 및 content 열 추가
    fieldnames = ["title", "link", "date", "content"]
    
    for row in reader:
        if "content" not in row:  # content 열이 없으면 빈 값으로 추가
            row["content"] = ""

    total_articles = len(reader)
    
    for start_idx in range(0, total_articles, batch_size):
        end_idx = min(start_idx + batch_size, total_articles)
        
        # tqdm으로 진행 상황 표시
        updated_articles = []
        
        for row in tqdm(reader[start_idx:end_idx], desc=f"Processing articles {start_idx + 1}-{end_idx}"):
            title = row.get("title", "").strip()
            link = row.get("link", "").strip()
            date = row.get("date", "").strip()
            
            # 이미 content가 있는 경우 건너뜀
            if "content" in row and row["content"].strip():
                validated_row = validate_row_fields(row, fieldnames)  # 필드 검증 추가
                updated_articles.append(validated_row)  # 기존 데이터 유지
                continue
            
            content = scrape_article_content(link)  # 기사 내용 크롤링
            
            # 기존 데이터에 content 추가 및 필드 검증 수행
            updated_row = {
                "title": title,
                "link": link,
                "date": date,
                "content": content,
            }
            validated_row = validate_row_fields(updated_row, fieldnames)  # 필드 검증 추가
            
            updated_articles.append(validated_row)
        
        # 출력 CSV 파일 저장 (덮어쓰기)
        with open(input_csv, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)  # 모든 필드를 따옴표로 감싸기
            
            writer.writeheader()  # 헤더 추가
            writer.writerows(reader[:start_idx] + updated_articles + reader[end_idx:])
        
        # 잠시 대기하여 서버 부하를 줄임
        time.sleep(5)

# 실행 코드
if __name__ == "__main__":
    input_csv_path = "cointelegraph_articles.csv"  # 입력 CSV 파일 경로
    
    update_csv_with_content_in_batches(input_csv=input_csv_path, batch_size=10)

Processing articles 1-10: 100%|██████████| 10/10 [00:18<00:00,  1.86s/it]


ValueError: dict contains fields not in fieldnames: None

In [ ]:
import pandas as pd
import re

# 1. 데이터 로드
data = pd.read_csv("cointelegraph_articles.csv")  # 크롤링된 데이터 파일

# 2. 텍스트 전처리 함수
def preprocess_text(text):
    """
    텍스트 전처리: HTML 태그 제거, 특수 문자 제거, 소문자 변환
    
    Parameters:
        text (str): 원본 텍스트
    
    Returns:
        cleaned_text (str): 정제된 텍스트
    """
    text = re.sub(r'<[^>]*>', '', text)  # HTML 태그 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    text = text.lower()  # 소문자 변환
    return text

# 3. 전처리 적용
data["cleaned_content"] = data["content"].apply(preprocess_text)

# 데이터 확인
print(data.head())

In [43]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze_sentiment_vader(content):
    """
    주어진 텍스트(content)의 감정을 분석하는 함수 (VADER 사용).
    
    Parameters:
        content (str): 분석할 텍스트
    
    Returns:
        sentiment (str): "Positive", "Negative", "Neutral" 중 하나
    """
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(content)
    
    # VADER의 compound 점수를 기준으로 감정 분류
    if sentiment_scores['compound'] >= 0.05:
        return "Positive"
    elif sentiment_scores['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

def perform_sentiment_analysis(input_csv, output_csv):
    """
    입력 CSV 파일에서 기사 내용을 읽어 감정 분석을 수행하고 결과를 새로운 CSV 파일에 저장하는 함수.
    
    Parameters:
        input_csv (str): 입력 CSV 파일 경로 (제목, 링크, 날짜, 내용 포함)
        output_csv (str): 출력 CSV 파일 경로 (제목, 링크, 날짜, 내용, 감정 포함)
    """
    articles_with_sentiment = []
    
    # 입력 CSV 파일 읽기
    with open(input_csv, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            title = row["title"]
            link = row["link"]
            date = row["date"]
            content = row["content"]
            
            # 감정 분석 수행
            sentiment = analyze_sentiment_vader(content)
            
            articles_with_sentiment.append({
                "title": title,
                "link": link,
                "date": date,
                "content": content,
                "sentiment": sentiment
            })
    
    # 출력 CSV 파일 저장
    with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
        fieldnames = ["title", "link", "date", "content", "sentiment"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        writer.writeheader()  # 헤더 추가
        writer.writerows(articles_with_sentiment)  # 모든 데이터를 한 번에 저장
    
    print(f"감정 분석 결과가 '{output_csv}' 파일에 저장되었습니다!")

# 실행 코드
if __name__ == "__main__":
    input_csv_path = "cointelegraph_articles_with_content.csv"  # 입력 CSV 파일 경로
    output_csv_path = "cointelegraph_articles_with_sentiment.csv"  # 출력 CSV 파일 경로
    
    perform_sentiment_analysis(input_csv=input_csv_path, output_csv=output_csv_path)

감정 분석 결과가 'cointelegraph_articles_with_sentiment.csv' 파일에 저장되었습니다!


In [8]:
import pandas as pd
import plotly.express as px

def process_sentiment_data(input_csv):
    """
    CSV 파일에서 데이터를 읽어와 날짜별 감정 강도를 처리하는 함수.
    
    Parameters:
        input_csv (str): 입력 CSV 파일 경로
    
    Returns:
        df_intensity (DataFrame): 날짜별 평균 감정 강도 데이터프레임
    """
    # CSV 파일 읽기
    df = pd.read_csv(input_csv)
    
    # 감정을 숫자로 매핑 및 가중치 적용
    sentiment_mapping = {
        "Positive": lambda x: 2 if "price rise" in x.lower() else 1.5,
        "Neutral": lambda x: 0,
        "Negative": lambda x: -2 if "price drop" in x.lower() else -1.5
    }
    
    # 각 기사 내용에 따라 가중치 적용
    df["sentiment_score"] = df.apply(
        lambda row: sentiment_mapping[row["sentiment"]](row["content"]), axis=1
    )
    
    # 날짜 형식 변환
    df["date"] = pd.to_datetime(df["date"])
    
    
    # 날짜별 평균 감정 강도 계산
    df_intensity = df.groupby("date")["sentiment_score"].mean().reset_index()
    
    return df_intensity

def visualize_sentiment_data(df_intensity):
    """
    Plotly를 사용하여 날짜별 감정 비율과 강도를 시각화하는 함수.
    
    Parameters:
        df_intensity (DataFrame): 날짜별 평균 감정 강도 데이터프레임
    """
    
    # 평균 강도 시각화
    fig_intensity = px.line(
        df_intensity,
        x="date",
        y="sentiment_score",
        title="날짜별 평균 감정 강도",
        labels={"date": "날짜", "sentiment_score": "평균 감정 점수"},
        markers=True
    )
    
    fig_intensity.update_layout(template="plotly_dark")
    fig_intensity.show()

# 실행 코드
if __name__ == "__main__":
    input_csv_path = "cointelegraph_articles_with_sentiment.csv"  # 입력 CSV 파일 경로
    
    # 데이터 처리 및 시각화
    df_intensity = process_sentiment_data(input_csv=input_csv_path)
    visualize_sentiment_data(df_intensity)

In [9]:
df = pd.read_csv('cointelegraph_articles.csv')
df

,title,link,date
0,XRP price poised for 46% gains after Ripple se...,https://cointelegraph.com/news/xrp-price-46-ga...,2025-03-14
1,Crypto regulation shifts as Bitcoin eyes $105K...,https://cointelegraph.com/news/crypto-regulati...,2025-03-13
2,Solana price bottom below $100? Death cross hi...,https://cointelegraph.com/news/solana-price-bo...,2025-03-13
3,Bitcoin price drops 2% as falling inflation bo...,https://cointelegraph.com/news/bitcoin-price-d...,2025-03-13
4,Trump family held talks with Binance for stake...,https://cointelegraph.com/news/trump-family-ha...,2025-03-13
...,...,...,...
112,Bitcoin rebounds to $84K — Analysts say BTC cr...,https://cointelegraph.com/news/bitcoin-rebound...,2025-03-01
113,BlackRock adds BTC ETF to model portfolio,https://cointelegraph.com/news/blackrock-adds-...,2025-03-01
114,"Price analysis 2/28: BTC, ETH, XRP, BNB, SOL, ...",https://cointelegraph.com/news/price-analysis-...,2025-03-01
115,Bitcoin price metric hits ‘optimal DCA’ zone n...,https://cointelegraph.com/news/bitcoin-price-h...,2025-03-01
